In [11]:
%run master_functions.ipynb

In [4]:
res = 'MET'

ihmm = file_rw.load_object('saved_parameters/2000iter_%s.pl' % res)['ihmm']

In [5]:
A, sigma, mu, T = organize_parameters(ihmm) # arrange parameters into arrays suitable for clustering

# file_rw.save_object({'A': A, 'sigma': sigma, 'mu': mu, 'T': T}, 'organized_params.pl')

tot_clusters = 30  # total clusters if combine_clusters = True

# clusters on each variable if combine_clusters = False
nclusters_A = 3
nclusters_sigma = 3
nclusters_r = 2
nclusters_T = 4

# total clusters on non-T variables if cluster_T_separate = True
nonT_clusters = 10

combine_clusters = True
cluster_T_separate = False
cluster_vars = 'eigs'
algorithm = 'agglomerative'

# default is diags
eigs = False
diags = True
if cluster_vars == 'eigs':
    eigs = True
    diags = False

In [10]:
nboot = 50  # number of bootstrap trials
frac = 0.4
ntraj = 100
dt = 0.5
endshow=2000
linkage='ward'  # possibilities: {“ward”, “complete”, “average”, “single”}
load = False
save = False

tot_clusters = [4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
tot_clusters = [20]
if load:
    
    final_parameters, trajectory_generators, clustered_labels = file_rw.load_object('clusters_%s_%s.pl' %(res, linkage))
    print('Loaded clusters_%s_%s.pl' % (res, linkage))
else:
    
    trajectory_generators = {}
    final_parameters = {}
    clustered_labels = {}

    for i, nC in enumerate(tot_clusters):

        sig_cluster, all_state_params = cluster_parameters(sigma, A, mu, T, combine_clusters, cluster_T_separate, 
                                                           eigs, diags, nC, nonT_clusters, nclusters_T,
                                                           nclusters_sigma, nclusters_r, nclusters_A, linkage=linkage)
        
        clustered_labels[nC] = sig_cluster.labels
        final_parameters[nC] = parameterize_clusters(sig_cluster, ihmm, show=False)
        print(np.linalg.norm(final_parameters[nC]['mu'][:, :2], axis=1))
        break
        nsteps = final_parameters[nC]['z'].shape[1]

        trajectory_generators[nC] = GenARData(params=final_parameters[nC])
        trajectory_generators[nC].gen_trajectory(nsteps, ntraj, bound_dimensions=[0, 1], resample_T=True)

    if save:
        
        file_rw.save_object((final_parameters, trajectory_generators, clustered_labels), 'clusters_%s_%s.pl' %(res, linkage))

Found 20 clusters



  0%|          | 0/10 [00:00<?, ?it/s]

Fitting 24 3 dimensional trajectories assuming an autoregressive order of 1



100%|██████████| 10/10 [00:29<00:00,  2.94s/it]


[0.7695 1.1585 0.1461 0.2416 1.135  1.0519 0.3922 0.5518 0.0946 4.4018
 0.3944 4.111  0.5102 2.9182 0.8172 1.6704 0.1421 3.1652 0.4584 0.8282]


## Visualize clustering on individual variables

In [ ]:
hist_limits = [[None, None, None, None], [None, None, [0, 10], None], [None, None, [0, 200], None]]
axis_limits = [[[0.5, 1], None],
               [[0, 10], None],
               [[0, 200], None]]

visualize_clusters(T, 'T', nclusters_T, eigs, diags, nbins=25, algorithm='agglomerative', linkage='ward', hist_limits=hist_limits, axis_limits=axis_limits)

In [ ]:
params = {'sigma': sigma, 'A': A, 'mu': np.square(np.linalg.norm(mu[:2, :], axis=0)), 'T': -np.log(1 - T)}
silhouette_(params, eigs, diags, 'ward', np.arange(2, 30), algorithm='agglomerative', plot=True, 
            savename='/home/ben/github/LLC_Membranes/Ben_Manuscripts/hdphmm/supporting_figures/silhouette_%s.pdf' % res)